In [4]:
pip install --upgrade transformers

Note: you may need to restart the kernel to use updated packages.


In [335]:
sys.version

'3.8.12 | packaged by conda-forge | (default, Jan 30 2022, 23:53:36) \n[GCC 9.4.0]'

In [8]:
pip install --upgrade pyfunctional

  Using cached PyFunctional-1.4.3-py3-none-any.whl (49 kB)
Note: you may need to restart the kernel to use updated packages.


In [308]:
%%capture
%run "context_learning_nrk.ipynb"

In [309]:
text1 = text
print(text1[0])

Instruction: Is this sentence correct. For example:
A should be easier to melt than B because B is hotter. Is that right? -> No



In [1]:
from IPython import get_ipython
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [13]:
import sys
import os

os.environ['HF_HOME'] = '/raid/xd/.cache/torch'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="7"
import random
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from types import MethodType
from tqdm import tqdm
from collections import defaultdict, OrderedDict, Counter
from datetime import datetime
from io import StringIO
from itertools import chain
import math
from functools import reduce
import numpy as np 
# np.set_printoptions(threshold=np.inf)
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data.sampler import RandomSampler, Sampler, SequentialSampler
from torch.utils.data.dataloader import DataLoader

from transformers.data.data_collator import DataCollator, default_data_collator
from transformers import AutoConfig, pipeline
from transformers import RobertaForMaskedLM, RobertaTokenizer, GPT2LMHeadModel, GPT2Tokenizer, GPTNeoForCausalLM, AutoModelForCausalLM
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import HfArgumentParser, Trainer, TrainingArguments, set_seed, AdamW
# from torch.optim.lr_scheduler import CosineAnnealingLR
# from transformers.trainer_utils import EvaluationStrategy


In [6]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPTNeoForCausalLM, GPTJForCausalLM

In [7]:
sys.path.insert(0, '/nas/xd/projects/PyFunctional')
from functional import seq
from functional.pipeline import Sequence
from fn import _
from collections import namedtuple 
import sys
# sys.path.insert(0, '/nas/xd/projects/transformers/src/transformers')
import os
# device_mappings = {0: 1, 1: 5, 2: 6, 3: 7, 4: 2, 5: 3, 6: 0, 1: 4}
# os.environ["CUDA_VISIBLE_DEVICES"] = str(device_mappings[2])

import random
import string
from collections import defaultdict, OrderedDict
from itertools import product, chain
import math
import numpy as np
from pattern.en import comparative
import types

In [348]:
sys.path.insert(0, '/nas/xd/projects/PyFunctional')
from functional import seq
from functional.pipeline import Sequence
from fn import _
from collections import namedtuple 
import sys
# sys.path.insert(0, '/nas/xd/projects/transformers/src/transformers')
import os

In [347]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss, MSELoss
import openai
openai.api_key = open('/nas/xd/projects/openai_api_keys.txt').readlines()[-1].split()[0]

In [338]:
n_total, n_valid =2,1
n_train = n_total - n_valid
# input_strs = [make_input_str(tasks[-7], nrows=1, ncols=(3,4,5)) for __ in range(n_total)]# Is the intersection of two sets empty.
# input_strs = [make_input_str(tasks[-3], nrows=1, ncols=(4,4,3)) for __ in range(n_total)]# Are there any elements different from other elements
# input_strs = [make_input_str(tasks[-4], nrows=1, ncols=(3,4,2,4)) for __ in range(n_total)]# Are_there_elements_belonging_to_the_same_class
# input_strs = [make_input_str(tasks[-1], nrows=1, ncols=(5,2)) for __ in range(n_total)]#Is_the_number_of_first_elements_greater_than_the_second_one
# input_strs = [make_input_str(tasks[-2], nrows=1, ncols=(3,1), ans_vocab=[0,1,2,3]) for __ in range(n_total)] #How_many_elements_are_similar_to_the_example
# input_strs = [make_input_str(tasks[-5], nrows=1, ncols=(3,4,2,4)) for __ in range(n_total)] #Are_they_the_union_of_the_last_element
# input_strs = [make_input_str(tasks[-6], nrows=1, ncols=(3,11,5)) for __ in range(n_total)]#Is the first set a subset of the second one
input_strs = [make_input_str(tasks[-10], nrows=1, ncols=(3, 3, 2, 5)) for __ in range(n_total)]
# input_strs = [make_input_str(tasks[-9], nrows=1, ncols=(4,2)) for __ in range(n_total)] #Compare the values of two sets.
# input_strs = [make_input_str(tasks[-8], nrows=1, ncols=(4, 4, 4)) for __ in range(n_total)] #Relationship between two sets.

for s in sample(input_strs, 2): print(s)

Instruction: Do all students choose courses in a department. For example:
D: Chinese, U: Art, A: Art, U: Chinese, D: Art; Art: Chi, Chinese: Eng, Physics: Eng. A and U,Chi? -> Yes

Instruction: Do all students choose courses in a department. For example:
C: Sql, H: Physics, H: Sql, H: Chemistry, K: Sql; Sql: Chi, Physics: Chi, Chemistry: Eng. H and C,Eng? -> No



In [339]:
def get_model_fn(engine):  # XD
    def fn(text):
        return openai.Completion.create(engine=engine, prompt=text, max_tokens=1, echo=True, logprobs=5).choices[0]
    return fn

engines = ['davinci', 'curie']
for engine in engines:
    model_name = 'openai_api_' + engine
#     model = lambda x: openai.Completion.create(engine=engine, prompt=x, max_tokens=0, echo=True, logprobs=5).choices[0]
    model = get_model_fn(engine)
    models[model_name] = model, tokenizer
    
# model_name = 'openai_api_davinci'
model_name = 'openai_api_curie'
model, tokenizer = models[model_name]

In [340]:
models.keys()

dict_keys(['EleutherAI/gpt-neo-1.3B', 'EleutherAI/gpt-j-6B', 'openai_api_davinci', 'openai_api_curie'])

In [341]:
def get_prob_dist(d, topk=5): return {k: round(math.exp(v), 3) for k, v in sorted(d.items(), key=lambda x: x[1], reverse=True)[:topk]}

In [342]:
def get_ans_indices0(input_ids):
    bos_id = tokenizer._convert_token_to_id('Ġ->')
    eos_id = tokenizer._convert_token_to_id('Ċ')
    bos_indices = (input_ids == bos_id).nonzero().squeeze(1).tolist()
    eos_indices = (input_ids == eos_id).nonzero()[-len(bos_indices):].squeeze(1).tolist()
    return bos_indices, eos_indices

# def get_ans_indices1(input_ids):
#     bos_id = tokenizer._convert_token_to_id('?')
# #     print("bos_id",bos_id)
#     eos_id = tokenizer._convert_token_to_id('Ċ')
#     period_ids = [tokenizer._convert_token_to_id('.'), tokenizer._convert_token_to_id('Ġ.')]
# #     print("period_ids:",period_ids)
# #     l_bracket_id, r_bracket_id = tokenizer._convert_token_to_id('Ġ['), tokenizer._convert_token_to_id('Ġ]')
# #     print("l_bracket_id",l_bracket_id)
# #     print("r_bracket_id:",r_bracket_id)
#     eos_indices = (input_ids == eos_id).nonzero().squeeze(1).tolist() #.nonezero()取非零元素坐标
# #     print("eos_indices0:",eos_indices)
#     eos_indices = [i - 1 if input_ids[i - 1] in period_ids else i for i in eos_indices]
# #     print("eos_indices1:",eos_indices)
# #     eos_indices = [i - 1 if input_ids[i - 1] == r_bracket_id else i for i in eos_indices]
# #     print("eos_indices2:",eos_indices)
    
#     def find_bos_index(start_i):
#         for bos_i in range(start_i, start_i - 3, -1):
# #             if input_ids[bos_i] == bos_id or input_ids[bos_i] == l_bracket_id and input_ids[bos_i - 1] == bos_id:
#             if input_ids[bos_i] == bos_id:
#                 if bos_i != start_i: print('subtokens:', tokenizer.convert_ids_to_tokens(input_ids[bos_i + 1: start_i + 2]))
#                 return bos_i
#         assert False
#     bos_indices = [find_bos_index(i - 2) for i in eos_indices]
# #     print("bos:",bos_indices)
# #     print("eos:",eos_indices)
#     return bos_indices, eos_indices

In [353]:
correct_num = 0
def predict(model, text, ans_indices_fn, topk=5, return_reduced_loss=False, verbose=True):
    use_openai_api = type(model) in [types.MethodType, types.FunctionType]  # openai.Completion.create
    print("[]",[types.MethodType, types.FunctionType])
    print("type(model):",type(model))
    inputs = tokenizer.encode_plus(text, return_tensors='pt')
    print(inputs)
    input_ids = inputs.input_ids
#     print(input_ids)
    bsz = input_ids.size(0)
#     print("bsz = ",bsz)
    labels = torch.ones_like(input_ids) * (-100)
    if use_openai_api:
        assert bsz == 1
#         outputs = model(engine=engine, prompt=text, max_tokens=0, echo=True, logprobs=5).choices[0].logprobs
        outputs = model(text).logprobs  # XD
        ans_nlls = []
    else:
        inputs = prepare_inputs(inputs, model.device)
        outputs = model(**inputs, output_attentions=False)
        logits = outputs.logits
    for bi in range(bsz):

        bos_indices, eos_indices = ans_indices_fn(input_ids[bi])
#         print("input_ids :",input_ids[bi])
#         print("ans_indices_fn:",ans_indices_fn(input_ids[bi]))
#         print("bos_indices:",bos_indices)
#         print("eos_indices:",eos_indices)
        examples = text.strip().split('\n')
#         print("bos_indices:",bos_indices)
        print("examples:",examples)
        assert len(bos_indices) == len(examples)-1, '%d != %d' % (len(bos_indices), len(examples))
        num = 0
        for i, (example, bos_i, eos_i) in enumerate(zip(examples, bos_indices, eos_indices)):
#             if verbose: print(' ' + example, end='\t')
            ans_ids = input_ids[bi, bos_i + 1: eos_i]
            print("ans_ids:",ans_ids)
            labels[bi, bos_i: eos_i - 1] = ans_ids
            if use_openai_api:
                ans_prob_dist = [get_prob_dist(d, topk=topk) for d in outputs.top_logprobs[bos_i + 1: eos_i]]
                ans_probs = [math.exp(lp) for lp in outputs.token_logprobs[bos_i + 1: eos_i]]
                ans_nlls += [-lp for lp in outputs.token_logprobs[bos_i + 1: eos_i]]
            else:
                ans_prob_dist = logits[bi, bos_i: eos_i - 1].softmax(-1)
                ans_probs = ans_prob_dist[torch.arange(ans_prob_dist.size(0)), ans_ids]
            ans_tokens = tokenizer.convert_ids_to_tokens(ans_ids)
            num += 1
            for ans_id, ans_token, ans_prob, dist in zip(ans_ids, ans_tokens, numpy(ans_probs, decimals=3), ans_prob_dist):
                top1_correct = max(dist.items(), key=lambda x: x[1])[0] == ans_token.replace('Ġ', ' ') \
                    if use_openai_api else (dist.argmax() == ans_id).item()  
                
                if verbose:
                    if(num == 1 and top1_correct):
                        global correct_num
                        correct_num += 1
                    if(len(ans_tokens) <= 1):
#                         if(top1_correct):
#                             dictlocation1[example] = 1
                        print(('！！！' if top1_correct else ' ') + ans_token, ans_prob, 
                              dist if use_openai_api else show_topk(*dist.topk(topk), indices_fn=tokenizer.convert_ids_to_tokens)) 
                        
                    else:
                        pass
    if use_openai_api:
        loss = ans_nlls if return_reduced_loss else sum(ans_nlls) / len(ans_nlls)
    else:
        loss = nn.CrossEntropyLoss()(logits.view(-1, logits.size(-1)), labels.view(-1)) if return_reduced_loss \
            else nn.CrossEntropyLoss(reduction='none')(logits.view(-1, logits.size(-1)), labels.view(-1))[labels.view(-1)>=0].tolist()
    return loss

In [354]:
print(len(input_strs))
print(input_strs[0])

2
Instruction: Do all students choose courses in a department. For example:
D: Chinese, U: Art, A: Art, U: Chinese, D: Art; Art: Chi, Chinese: Eng, Physics: Eng. A and U,Chi? -> Yes



In [355]:
correct_num = 0
for i in input_strs:
# for i in text1:
    text = i
    predict(model, text, get_ans_indices0, verbose=True, topk=5)

[] [<class 'method'>, <class 'function'>]
type(model): <class 'function'>
{'input_ids': tensor([[ 6310,  2762,    25,  2141,   477,  2444,  3853, 10902,   287,   257,
          5011,    13,  1114,  1672,    25,   198,    35,    25,  3999,    11,
           471,    25,  3683,    11,   317,    25,  3683,    11,   471,    25,
          3999,    11,   360,    25,  3683,    26,  3683,    25, 21380,    11,
          3999,    25,  1985,    11, 23123,    25,  1985,    13,   317,   290,
           471,    11,  1925,    72,    30,  4613,  3363,   198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
examples: ['Instruction: Do all students choose courses in a department. For example:', 'D: Chinese, U: Art, A: Art, U: Chinese, D: Art; Art: Chi, Chinese: Eng, Physics: Eng. A and U,Chi? -> Yes']
ans_ids: tensor([3363])
 ĠYes 

4.8359756

[] [<class 'method'>, <class 'function'>]
type(model): <class 'function'>
{'input_ids': tensor([[ 6310,  2762,    25,  2141,   477,  2444,  3853, 10902,   287,   257,
          5011,    13,  1114,  1672,    25,   198,    34,    25,   311, 13976,
            11,   367,    25, 23123,    11,   367,    25,   311, 13976,    11,
           367,    25, 27867,    11,   509,    25,   311, 13976,    26,   311,
         13976,    25, 21380,    11, 23123,    25, 21380,    11, 27867,    25,
          1985,    13,   367,   290,   327,    11,  7936,    30,  4613,  1400,
           198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
examples: ['Instruction: Do all students choose courses in a department. For example:', 'C: Sql, H: Physics, H: Sql, H: Chemistry, K: Sql; Sql: Chi, Physics: Chi, Chemistry: Eng. H and C,E

3.990212

# 从这里开始

In [8]:
from utils import *
from child_utils import *
from common_utils import *
from model_utils import *

In [10]:
models = {}
cache_dir = '/nas/xd/.cache/torch/transformers/'  # for models besides t5-3b/11b
proxies = {'http': '192.168.50.1:1081'} 

In [14]:
model_name = "EleutherAI/gpt-j-6B"
# model_name = "EleutherAI/gpt-neo-1.3B"
# model = GPTJForCausalLM.from_pretrained(model_name, proxies=proxies, cache_dir=cache_dir)
model = AutoModelForCausalLM.from_pretrained(model_name, proxies=proxies, cache_dir=cache_dir)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', cache_dir=cache_dir)
models[model_name] = model, tokenizer

In [15]:
_ = model.eval()
unify(model)
blocks = model.transformer.h
attn = blocks[0].attn
ln_f = model.transformer.ln_f
L, H, embed_dim = len(blocks), attn.num_heads, attn.embed_dim

_we = model.transformer.wte.weight.data.t()
_wu = model.lm_head.weight.data

In [23]:
_wu.size()

torch.Size([50400, 4096])

In [16]:
def T(f): return lambda x: f(x.T).T

In [53]:
layer, head = 3, 1
wq, wk, wv, wo = get_head_weights(model, layer, head)
mlp = lambda x: mlp_forward(blocks[0], x)
ln = blocks[layer].ln_1
v = _we.size(1) #// 2
we, wu = _we[:, :v], _wu[:v]

In [54]:
qk = True
with torch.no_grad():  # ve,ed,de,ev->vv
    _m = ln(mlp(we.T)) @ (wq.T @ wk) @ T(ln)(T(mlp)(we)) if qk else wu @ T(ln_f)(wo @ wv @ T(ln)(T(mlp)(we)))

In [55]:
m = _m  # q->k, output->input
# m = _m.T  # k->q, input->output
values, indices = m.topk(10)

In [56]:
show_topk.indices_fn = tokenizer.convert_ids_to_tokens
for i in values.mean(1).topk(1000).indices.tolist():
    print(tokenizer.convert_ids_to_tokens(i), show_topk(values[i].long(), indices[i]))

Ġever {'ottest': 65, 'Ġdarkest': 59, 'ĠGreatest': 58, 'Ġholiest': 58, 'Ġtallest': 57, 'Ġtoughest': 56, 'Ġbrightest': 55, '}}}': 55, 'Ġdeadliest': 54, 'liest': 54}
Ġso {'}}}': 62, 'Ġ}}': 60, '}}': 59, 'Ġ--------------------': 59, '))))': 58, 'ottest': 55, ')}': 54, '---------------': 54, ')]': 54, 'Ġ)))': 54}
ĠEVER {'ottest': 63, 'ĠGreatest': 61, 'Ġbrightest': 57, 'Ġdarkest': 57, 'Ġtallest': 57, 'Ġtoughest': 55, 'Ġcoolest': 55, 'Ġdeadliest': 55, 'liest': 53, 'gest': 53}
Ġjealous {'ĠMaking': 59, 'ĠMakes': 58, 'Ġmaking': 57, 'Ġmakes': 57, 'ĠMake': 57, 'Ġmade': 56, 'Making': 55, 'ĠMade': 54, 'Made': 54, 'Ġmake': 52}
ĠEver {'ottest': 62, 'ĠGreatest': 59, 'Ġtallest': 58, 'Ġdarkest': 57, 'Ġholiest': 56, 'Ġtoughest': 54, 'Ġbrightest': 53, 'Ġhottest': 52, 'Ġbiggest': 52, 'liest': 51}
Ġamong {'Ġcirculated': 61, 'Ġuniqueness': 60, 'Ġinconsistencies': 59, 'iqueness': 58, 'Ġbestselling': 55, 'Ġpercentile': 53, 'Ġnumbered': 53, 'ĠIncluded': 52, 'unique': 52, 'Ġdistributor': 52}
Ġevery {'>]': 60, '}}

Ġgrants {'}}}': 48, '))))': 47, ')))': 45, '})': 44, 'Ġ)))': 43, '}}': 43, 'Ġ}}': 42, '])': 42, '))': 40, ']}': 40}
ĠThan {'ĠMORE': 50, 'Ġsooner': 48, 'MORE': 47, 'Ġnewer': 42, 'Ġsmoother': 42, 'more': 41, 'ĠMore': 41, 'Ġhotter': 41, 'Ġhealthier': 41, 'Ġhappier': 41}
ĠOnce {'}}}': 48, ']]': 47, '))))': 47, 'Ġ}}': 46, '>]': 44, ']}': 43, '}}': 41, ']=': 40, "']": 39, 'Ġ)))': 39}
ever {'}}}': 47, 'ottest': 45, 'Ġ}}': 44, '="': 44, '"/>': 43, '))))': 43, '},{"': 42, 'Ġtallest': 42, '=\\"': 42, 'Ġ)))': 42}
asses {'}}}': 49, '))))': 47, 'Ġ}}': 44, '}}': 44, ')))': 44, 'Ġ)))': 43, '>]': 43, '})': 42, '"))': 40, 'Ġ))': 40}
ĠFranken {'ĠChanged': 47, 'ĠIncreased': 45, '},"': 44, '(_': 44, 'Changed': 44, '>(': 43, '}}}': 42, 'Increased': 42, 'ĠProject': 42, '=\\"': 41}
ĠJudge {'}}}': 50, '}}': 46, 'Ġ}}': 44, '>]': 44, '))))': 43, '},{"': 43, ']}': 42, '"}],"': 41, '})': 41, '},"': 41}
Ġobvious {'}}}': 48, '})': 46, '))))': 45, '}}': 44, 'Ġ}}': 44, ')))': 42, 'Ġ)))': 41, '"))': 41, '))': 41, 'Ġ})

In [47]:
i = tokenizer.encode(' June')[0]; print(tokenizer.convert_ids_to_tokens(i))
show_topk(*m[i].topk(20))

ĠJune


{'ĠMay': 48.225,
 'May': 40.55,
 'ĠApril': 33.73,
 'ĠMAY': 33.482,
 'ĠApr': 32.515,
 'Ġapr': 29.668,
 'April': 29.434,
 'Ġhiber': 29.343,
 'ĠEaster': 28.028,
 'ĠHait': 27.919,
 'Apr': 27.211,
 'ĠHamp': 26.851,
 'ĠMarch': 26.381,
 'ĠAlph': 23.626,
 'March': 23.49,
 'Ġmasc': 23.334,
 'ĠFebruary': 23.255,
 '05': 22.955,
 'may': 22.247,
 'ĠAPR': 21.985}

In [9]:
prompt_token = 'Ġ!'; prompt_id = tokenizer._convert_token_to_id(prompt_token)
bop_str = 'Instruction: '; bop_id = tokenizer.encode(bop_str)[0]  # 'Inst'
eop_str = '. For example:'; eop_id = tokenizer.encode(eop_str)[2] # 'Ġexample'
bos_id = tokenizer._convert_token_to_id('Ġ->')
eos_id = tokenizer._convert_token_to_id('Ċ')


class CHILDDataset(Dataset):
    def __init__(self, input_strs, tokenizer):
        if tokenizer.pad_token is None: tokenizer.pad_token = '!'
        self.inputs = tokenizer.batch_encode_plus(input_strs, add_special_tokens=False, padding=True, return_tensors='pt')#长的截，短的补
        input_ids = self.inputs.input_ids
        self.labels = torch.ones_like(input_ids) * (-100)

        for bi in range(input_ids.size(0)): 
            bop_idx = (input_ids[bi] == bop_id).nonzero().squeeze(1) #prompt
            eop_idx = (input_ids[bi] == eop_id).nonzero().squeeze(1) #context

            if len(bop_idx) > 0:
                assert len(bop_idx) == 1 and len(eop_idx) == 1
                bop_idx, eop_idx = bop_idx.item(), eop_idx.item() #取出单元素张量的元素值并返回该值，保持原元素类型不变
                #bop: 0   eop:6
                input_ids[bi, bop_idx: eop_idx + 2] *= -1  # use prompt embedding for prompt tokens
  
            bos_indices = (input_ids[bi] == bos_id).nonzero().squeeze(1)
#             print("bos_indices:",bos_indices)
            eos_indices = (input_ids[bi] == eos_id).nonzero()[-len(bos_indices):].squeeze(1) #每一位 eos都比bos大2
#             print("eos_indices:",eos_indices)
            for i, (bos_i, eos_i) in enumerate(zip(bos_indices.tolist(), eos_indices.tolist())):
                assert eos_i > bos_i + 1
                if i >= 0:  #zero-shot
                    self.labels[bi, bos_i + 1: eos_i] = input_ids[bi, bos_i + 1: eos_i] 
        
        
    def re_input(self):
        return self.inputs['input_ids']
        
    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, i):
        return {'input_ids': self.inputs['input_ids'][i],  #输入
                'attention_mask': self.inputs['attention_mask'][i],
                'labels': self.labels[i]}   #结果

In [10]:
bos_id,eos_id

(4613, 198)

In [11]:
from child_utils import *
torch.set_printoptions(threshold=np.inf)

In [12]:
class WrappedEmbedding(nn.Module):
    def __init__(self,  
                wte: nn.Embedding,  #正常向量
                prompt_id: int = None,
                prompt_len: int = 10, 
                random_range: float = 0.5,
                initialize_from_vocab: bool = True):
        super(WrappedEmbedding, self).__init__()
#         self.wte = wte
#         self.prompt_id = prompt_id
#         self.prompt_len = prompt_len
        self.__dict__.update(locals()); del self.self #locals()以字典类型返回当前位置的全部局部变量
        if self.prompt_id is not None: #prompt_embedding prompt词向量
            self.prompt_embedding = nn.parameter.Parameter( #将一个不可训练的类型Tensor转换成可以训练的类型parameter
                self.initialize_embedding(random_range, initialize_from_vocab)).to(self.wte.weight.device) #在-0.5-0.5中随机取值初始化
        else:
            self.prompt_embedding = nn.Embedding(self.prompt_len, self.wte.weight.size(1)).to(self.wte.weight.device)
                                        #词典大小（总共输入多少词） 嵌入向量维度（多少维表示一个符号）
            
            assert initialize_from_vocab
            self.init_prompt_embedding_()  #将wte的weight值作为初始化
#             self.prompt_embedding.weight.data = self.initialize_embedding(random_range, initialize_from_vocab)     
 
    def initialize_embedding(self, random_range: float = 0.5, initialize_from_vocab: bool = True):
        if initialize_from_vocab: return self.wte.weight[:self.prompt_len].clone().detach() #返回一个新的tensor，新的tensor和原来的tensor共享数据内存，但不涉及梯度计算
        return torch.FloatTensor(self.prompt_len, self.wte.weight.size(1)).uniform_(-random_range, random_range) #产生随机数
    
    def init_prompt_embedding_(self):
#         print(self.wte.weight)
        self.prompt_embedding.weight.data[:] = self.wte.weight[:self.prompt_len]

        
    def forward(self, input_ids):
        if self.prompt_id is not None:
            input_embeds = self.wte(input_ids)
            input_embeds[input_ids == self.prompt_id] = self.prompt_embedding.expand(input_embeds.size(0), -1, -1)
        else: # adapted from cpm-2
            prompt_mask = input_ids < 0  #返回bool类型
#             print("prompt_mask:",prompt_mask.shape)
            prompt_ids = -input_ids * prompt_mask #将prompt的值变为正数，其他置零
#             print("prompt_ids:",prompt_ids)
#             print(prompt_ids < self.prompt_len)
#             print(prompt_ids)
            assert torch.all(prompt_ids < self.prompt_len)
#             print(self.prompt_embedding(prompt_ids).shape)
            p_embeds = self.prompt_embedding(prompt_ids) * prompt_mask.float().unsqueeze(-1)
#             print("p_embeds:",p_embeds.shape)
            input_ids = input_ids * ~prompt_mask
            w_embeds = self.wte(input_ids) * (~prompt_mask).float().unsqueeze(-1)
#             print("w_embeds:",w_embeds.shape)
            input_embeds = w_embeds + p_embeds 
#         print(input_embeds)
        return input_embeds

In [13]:
# adapted from cpm-2: https://github.com/TsinghuaAI/CPM-2-Finetune/blob/master/utils.py#L133-L164    #不训练模型参数，只训练prompt_embading，这个函数是取这些参数
def get_params_for_prompt_optimization(module: nn.Module): 
    params = []
    for t in module.named_modules():
        if "prompt_embedding" in t[0]:
            params.append({'params': [p for p in list(t[1]._parameters.values()) if p is not None]})
    for t in module.named_parameters():
        if "prompt" not in t[0]:
            t[1].requires_grad_(False)    
    return params

def create_optimizer(model, training_args):
    from torch.nn.parallel.distributed import DistributedDataParallel as DDP
    while isinstance(model, (DDP, )): model = model.module
        
    we.init_prompt_embedding_()
    param_groups = get_params_for_prompt_optimization(model)
    optimizer = AdamW(param_groups, lr=training_args.learning_rate, 
                      betas=(training_args.adam_beta1, training_args.adam_beta2),eps=training_args.adam_epsilon)
    return optimizer

In [14]:
# wte = model.get_input_embeddings() #提取 embedding 中的 词向量部分
# if hasattr(wte, 'wte'): wte = wte.wte  # already been wrapped
# we = WrappedEmbedding(wte, prompt_len=20000)
# model.set_input_embeddings(we) #为词向量赋值

In [15]:
def verbalize(obj):
    if type(obj) == bool: return 'Yes' if obj else 'No'
    return str(obj)

def list2str(l): return ' '.join(str(i) for i in l)
def pairs2str(pairs): return ', '.join(str(k) + ': ' + str(v) for k, v in pairs)

def make_context_str(cxt):
    if type(cxt) == list:
        return pairs2str(cxt) if type(cxt[0]) == tuple and len(cxt[0]) == 2 else list2str(cxt)
    if type(cxt) == tuple:
        return '; '.join(make_context_str(c) for c in cxt)  # 用分号分隔context的不同部分
    
def make_query_str(instruction, query):
    if instruction is None and query is None: return ''
    s = '.'
    if instruction is not None: s = s + ' ' + instruction
    if query is not None:
        if type(query) in [int, bool, str]: query = [query]
        if type(query) == dict:# and list(query.keys())[0] != "CS"):  # by nrk
            s = s + ' ' + '{' + ','.join([' replace %s with %s' % (str(k), str(v)) for k, v in query.items()]) + ' }'
        if type(query) in [list, tuple]:
            s = s + ' ' + ' '.join([list2str(i) if type(i) == list else str(i) for i in query])
    return s

def make_example_str(example, query2str):
    instruction, cxt, query, ans = example
    if type(ans) not in [Sequence, list]: ans = [ans]
    ans = [verbalize(a) for a in ans]
#     return '%s -> %s' % (''.join(l[0]) + make_query_str(instruction if with_instruction else None, query), ' '.join(ans))  # by nrk
#     return '%s -> %s' % (' '.join(l) + make_query_str(instruction if with_instruction else None, query), ' '.join(ans))  # by XD
#     return '%s -> %s' % (make_context_str(cxt) + make_query_str(instruction if with_instruction else None, query[0]), ' '.join(ans))
    return '%s -> %s' % (make_context_str(cxt) + query2str(query), ' '.join(ans))


def sample_rand_len(vocab, k): return sample(vocab, k=randint(1, k))

In [27]:
def promptize(s):
#     return prompt_token * len(s.split())
    return bop_str + s + eop_str

courses_vocab=["Sql","Math","English","Chinese","Art","Music","History","Biology","Chemistry","Physics","Geography"]
all_vocab = ["apple","pear","peach","grape","banana","pineapple","lemon","strawberry","cola","juice","coffee","milk","wine","beer","whisky","vodka","brandy"]
solid_vocab=["apple","pear","peach","grape","banana","pineapple","lemon","strawberry"]
liquid_vocab=["cola","juice","coffee","milk","wine","beer","whisky","vodka","brandy"]
names_vocab =  [i for i in string.ascii_uppercase]
depts_vocab = ["CS","Chi","Eng","Mat"]
sl = ["solid"]*len(solid_vocab)+["liquid"]*len(liquid_vocab)
sl_vocab = dict(zip(all_vocab,sl))
toys = ["ball","doll","puppet","weiqi","chess","slide","diabolo","plasticine","tumbler"]
                                              #滑梯     空竹      橡皮泥      不倒翁
boys = ["Alex","Dylan","Daniel","Patrick","Austin","Harrison","Tom","Neil"]
girls = ["Ashley","jessica","Sarah","Amanda","Nicole","Katherine","Anne","Eva"]
all = boys+girls
bg = ["boys"]*len(boys)+["girls"]*len(girls)
bg_vocab = dict(zip(all,bg))

all = solid_vocab+toys
ft = ["fruits"]*len(solid_vocab)+["toys"]*len(toys)
ft_vocab = dict(zip(all,ft))

def make_input_str(task, nrows=4, ncols=4, full_vocab=None, ans_vocab=[True, False]):
    if full_vocab is None: full_vocab = string.ascii_uppercase + string.digits
    transform_fn, vocab_fn, sample_fn, query_fn, query2str = task
    instruction = transform_fn.__name__.replace('_', ' ')
    if vocab_fn is None: vocab_fn = lambda: full_vocab
    if query_fn is None: query_fn = lambda *_: None
        
    examples = []
    query = None
    for i in range(nrows):
        vocab = vocab_fn()
        l = sample_fn(vocab, k=ncols)
        query = query_fn(l, vocab, ncols)
        examples.append([instruction, l, query, transform_fn(l, query=query)])
#     examples = balance(examples,ans_vocab)

    desc = promptize(instruction) if True else ''
    text = '\n'.join([make_example_str(e, query2str) for e in examples])
    text = desc + '\n' + text + '\n'
    return text

In [17]:
# # def balance(examples, ans_vocab=[True, False]):
# def balance1(examples, ans_vocab):
#     groups = seq(examples).group_by(_[-1]).map(_[1])  # 按ans分组
# #     assert groups.len() == len(ans_vocab), '%d < %d' % (groups.len(), len(ans_vocab))  # 保证每种ans都出现
#     min_cnt = groups.map(lambda x: len(x)).min()
#     examples = groups.map(lambda x: sample(x, min_cnt)).flatten().list() # 每组都采样最小个数后去分组
#     return sample(examples, len(examples))  # 重新打乱

In [18]:
def balance(examples, ans_vocab):
    groups = seq(examples).group_by(_[-1]).map(_[1])  # 按ans分组
#     min_cnt = groups.map(lambda x: len(x)).min()
    min_cnt =3
    if(groups.len()>3):
        min_cnt = 3
    elif(groups.len()==3):
        min_cnt = 3
    if(min_cnt > 2):
        examples = groups.map(lambda x: sample(x, 3)).flatten().list() # 每组都采样最小个数后去分组
        return sample(examples, len(examples))  # 重新打乱
    else:
        examples = groups.map(lambda x: sample(x, min_cnt)).flatten().list() # 每组都采样最小个数后去分组
        return sample(examples, len(examples))  # 重新打乱

In [19]:
import itertools  
def Do_all_students_choose_courses_in_a_department(cxt, query):
    SC, CD = cxt  # SC paris: studeng-course relation, CD pairs: course-department function
    ss, d = query  # ss: 学生子集（可以*不止两个学生*），d: 课程
#     return seq(ss).map(lambda s: seq(SC).filter(_[0] == s).map(_[1]).intersection(CD.filter(_.[1] == d).map(_.[0])).non_empty()).all()
    return (seq(ss)
            .map(lambda s: seq(SC).filter(_[0] == s).map(_[1])  # 学生s选的所有课程
                 .intersection(
                     seq(CD).filter(_[1] == d).map(_[0])) # d系的课程
                 .non_empty())  # s选了d系的课程
            .all())  # 学生子集ss都选了d系的课程

def all_a_sample(vocab, k):
    S_vocab, C_vocab, D_vocab = vocab  # vocabs of students, courses, departments
    k_S, k_C, k_D, k_SC = k  # default values: k_S = 3, k_C = 3, k_D = 2, k_SC = 5
    S, C, D = sample(S_vocab, k_S), sample(C_vocab, k_C), sample(D_vocab, k_D)
    
    while len(set(CD := choices(D, k=k_C))) < k_D: continue  # ds里每个系的课都要出现
    CD = list(zip(C, CD))  # 得到每门课所属的系
    
    all_SC = list(itertools.product(S, C))  # or seq(S).cartesian(C).list()
    while seq(SC := sample(all_SC, k_SC)).map(_[0]).distinct().len() < k_S: continue  # ss里每个学生都要选课
    return SC, CD

def select_distinct(tuples, col): return seq(tuples).map(_[col]).distinct().list()
    
def all_a_query(cxt,vocab,k):
    SC, CD = cxt
    k_S, k_C, k_D, k_SC = k
    S, D = select_distinct(SC, 0), select_distinct(CD, 1)
#     k_ss = randint(2, len(S))
    ss = sample(S, 2)
    d = choice(D)
    return ss, d

def all_a_query2str(query):
    ss, d = query
    query_str = '%s,%s?' % (', '.join(ss[:-1]) + ' and ' + ss[-1], d)
    return '. ' + query_str


In [20]:
def all_b(cxt, query):
    SC, CD = cxt
    ss,d = query
    return (seq(CD).filter(_[1] == d).map(_[0])
                 .difference(
                     seq(SC).filter(_[0] == ss).map(_[1]))
                 .empty())

def all_b_sample(vocab, k):
    S_vocab, C_vocab, D_vocab = vocab  
    k_S, k_C, k_D, k_SC = k  # default values: k_S = 2, k_C = 3, k_D = 2, k_SC = 5
    S, C, D = sample(S_vocab, k_S), sample(C_vocab, k_C), sample(D_vocab, k_D)
    
    while len(set(CD := choices(D, k=k_C))) < k_D: continue  
    CD = list(zip(C, CD)) 
    
    all_SC = list(itertools.product(S, C))  
    while seq(SC := sample(all_SC, k_SC)).map(_[0]).distinct().len() < k_S: continue  # ss里每个学生都要选课
    return SC, CD
   
def all_b_query(cxt,vocab,k):  # XD: 不要给qeury_fn加st参数
    SC, CD = cxt
    k_S, k_C, k_D, k_SC = k
    S, D = select_distinct(SC, 0), select_distinct(CD, 1)
#     k_ss = randint(2, len(S))  # XD: k_ss unused
    ss = choice(S)
    d = choice(D)
    return ss, d
    # XD: 不要在query_fn里转str！！这里转str，transform_fn里再解析回来，两边不是白折腾吗！

def all_b_query2str(query):
    ss, d = query
    query_str = 'Does %s take all %s courses?' % (ss, d)
    return '. ' + query_str


def Is_the_intersection_of_two_sets_empty(cxt, query):
    SC= cxt
    s,d = query
    return (seq(SC).filter(_[0] == s).map(_[1])
                 .intersection(
                     seq(SC).filter(_[0] == d).map(_[1]))
                 .non_empty())

def intersection_query(cxt,vocab,k):
    SC= cxt
    k_S, k_C, k_SC = k #k_s = 3, k_C = 5 , k_sc = 6
    S = select_distinct(SC, 0)
    s,d = sample(S,2)
    return s, d

def intersection_query2str(query):
    s, d = query
    query_str = "%s,%s?" % (s, d)
    return '. ' + query_str

def intersection_sample(vocab, k):
    S_vocab, C_vocab = vocab  
    k_S, k_C, k_SC = k  # k_S = 3, k_C = 5, , k_SC = 6
    S, C= sample(S_vocab, k_S), sample(C_vocab, k_C)
    all_SC = list(itertools.product(S, C))  
    while seq(SC := sample(all_SC, k_SC)).map(_[0]).distinct().len() < k_S: continue 
    return SC

def Is_the_first_set_a_subset_of_the_second_one(cxt, query):
    SC= cxt
    s,d = query
    return (seq(SC).filter(_[0] == d).map(_[1])
                 .union(
                     seq(SC).filter(_[0] == s).map(_[1]))
                 ).distinct().len()== seq(SC).filter(_[0] == d).map(_[1]).distinct().len()

def complement_query(cxt,vocab,k):
    SC= cxt
    k_S, k_C, k_SC = k #k_s = 3, k_C = 5 , k_sc = 6
    S = select_distinct(SC, 0)
    s,d = sample(S,2)
    return s, d

def complement_query2str(query):
    s, d = query
    query_str = "%s,%s?" % (s, d)
    return '. ' + query_str

def complement_sample(vocab, k):
    S_vocab, C_vocab = vocab  
    k_S, k_C, k_SC = k  # k_S = 3, k_C = 4, k_SC = 5
    S, C= sample(S_vocab, k_S), sample(C_vocab, k_C)
    all_SC = list(itertools.product(S, C))  
    while seq(SC := sample(all_SC, k_SC)).map(_[0]).distinct().len() < k_S: continue 
    return SC

def Are_they_the_union_of_the_last_element(cxt, query):
    SC, DC = cxt
    ss,d = query
    return (seq(ss)
            .map(lambda s: seq(SC).filter(_[0] == s).map(_[1]) 
                 .union(
                     seq(DC).filter(_[0] == d).map(_[1])) 
                 .distinct().len() == seq(DC).filter(_[0] == d).map(_[1]).distinct().len())  
            .all()) 

def union_sample(vocab, k):
    S_vocab, C_vocab, D_vocab = vocab  
    k_S, k_C, k_D, k_SC = k  # k_S = 3, k_C = 4, k_D = 2, k_SC = 6
    S, C, D = sample(S_vocab, k_S), sample(C_vocab, k_C), sample(D_vocab, k_D)
    
    while len(set(DC := choices(D, k=k_C))) < k_D: continue  
    DC = list(zip(DC,C)) 
    
    all_SC = list(itertools.product(S, C))  
    while seq(SC := sample(all_SC, k_SC)).map(_[0]).distinct().len() < k_S: continue  
    return SC, DC
   
def union_query(cxt,vocab,k): 
    SC, DC = cxt
    k_S, k_C, k_D, k_SC = k
    S, D = select_distinct(SC, 0), select_distinct(DC, 0)
    k_ss = randint(2, len(S))
    ss = sample(S, k_ss)
    d = choice(D)
    return ss, d

def union_query2str(query):
    ss, d = query
    query_str = '%s,%s?' % (', '.join(ss[:-1]) + ' and ' + ss[-1], d)
    return '. ' + query_str

def Are_there_elements_belonging_to_the_same_class(cxt, query): #新建联系，把vocab和属性连起来形成新集合
    NA, DA = cxt
    s,d = query
    D = seq(DA).filter(_[0] == d).map(_[1])
    return (seq(NA).filter(_[0] == s).map(_[1]).select(lambda x: sl_vocab[x] == sl_vocab[D[0]]).any())
      
    
def find_sample(vocab, k): 
    name_vocab, all_vocab = vocab
    k_name, k_all, k_D, k_SA = k  # k_name = 3, k_all = 4, k_D = 2, k_SA = 6
    S = list(all_vocab.keys())
    N, A, D = sample(name_vocab, k_name), sample(S, k_all), sample(string.ascii_lowercase, k_D)
#     D = sample(d := name_vocab.pop(name_vocab.index(a) for a in N),k_D)
    A1 = sample(S, k_D)
    DA = list(zip(D,A1)) 
    
    all_NA = list(itertools.product(N, A))  
    while seq(NA := sample(all_NA, k_SA)).map(_[0]).distinct().len() < k_name: continue  
    return NA, DA
   
def find_query(cxt,vocab,k): 
    NA, DA = cxt
    k_name, k_all, k_D, k_SA = k
    S,D = select_distinct(NA, 0), select_distinct(DA, 0)
    s,d = choice(S), choice(D)
    return s, d

def find_query2str(query):
    s, d = query
    query_str = '%s,%s?' % (s, d)
    return '. ' + query_str


def Are_there_any_elements_different_from_other_elements(cxt, query):
    NA = cxt
    ss = query
    return (seq(ss).map(lambda s: seq(NA).filter(_[0] == s).map(_[1])
                        .select(lambda x: sl_vocab[x])[0])
            .distinct().len( ) == 2)
               
def find_dif_sample(vocab, k):
    name_vocab, all_vocab = vocab
    k_name, k_all, k_SA = k  # k_name = 4, k_all = 4, k_SA= 3
    N, A = sample(name_vocab, k_name), sample(list(all_vocab.keys()), k_all) 
    NA = list(zip(N,A)) 
    return NA
   
def find_dif_query(cxt,vocab,k): 
    NA = cxt
    k_name, k_all, k_NA = k
    S = select_distinct(NA, 0)
    ss = sample(S,k_NA)
    return ss


def find_dif_query2str(query):
    ss = query
    query_str = '%s?' % (', '.join(ss[:-1]) + ' and ' + ss[-1])
    return '. ' + query_str



In [21]:
def How_many_elements_are_similar_to_the_case(cxt, query):
    s = cxt
    q = query
    return seq(s).filter(lambda x: sl_vocab[x] == sl_vocab[q[0]]).len()
                     
def count_sample(vocab, k):
    all_vocab = vocab
    k_cxt,k_query = k  # k_cxt = 4 ,k_query =1
    N = list(all_vocab.keys())
    s = sample(N, k_cxt) 
    return s
   
def count_query(cxt,vocab,k): 
    NA = cxt
    k_cxt,k_query = k
    N = list(vocab.keys())
    q = sample(N,k_query)
    return q

def count_query2str(query):
    q = query
    query_str = '%s?' % (q[0])
    return '. ' + query_str

def Is_the_number_of_first_elements_greater_than_the_second_one(cxt, query):
    s = cxt
    q = query
    return (seq(s).filter(lambda x: sl_vocab[x] == sl_vocab[q[0]]).len() >= len(s)/2)
                     
def compare_sample(vocab, k):
    all_vocab,s_vocab,l_vocab = vocab
    k_cxt,k_query = k  # k_cxt = 5 ,k_query =2
    N = list(all_vocab.keys())
    s = sample(N, k_cxt) 
    return s
   
def compare_query(cxt,vocab,k): 
    NA = cxt
    all_vocab,s_vocab,l_vocab = vocab
    k_cxt,k_query = k
    q = sample(s_vocab,1)+sample(l_vocab,1)
    random.shuffle(q)
    return q

def compare_query2str(query):
    ss,q = query
    query_str = '%s,%s?' % (ss,q)
    return '. ' + query_str

In [22]:
def Ascii_size_existence(l, query): return seq(l).map(_[0] > query).any()
def Ascii_size_all(l, query): return seq(l).map(_[0] > query).all()
def Ascii_size_None(l, query): return seq(l).filter(_[0] > query).empty()
def ith_element(l, query=None): return seq(l).slice(1, 2)
def ith_group(l, query=None): return seq(l).group_by(_).select(_[1]).slice(1, 2).flatten()#.distinct()# davinci F w/ and wo dist
# def element_at_index(l, query): return seq(l).slice(query, query + 1) # davinci F
def element_at_index(l, query): return seq(l).enumerate().filter(_[0] == query).select(_[1])
def replace(l, query): return seq(l).map(lambda x: query.get(x, x))
def replace_with_the_other(l, query): # davinci F
    query = {k: (set(l) - {k}).pop() for k in l}
    return replace(l, query)
def replace_all_with(l, query): return seq(l).map(lambda x: query)  # davinci F?!
def interleave_with(l, query): return seq(l).flat_map(lambda x: [x, query])  # davinci T!!
def unique_elements(l, query=None): return seq(l).distinct() # davinci F
def how_many_unique_elements(l, query=None): return seq(l).distinct().len()  # davinci F
def how_many(l, query): return seq(l).filter(_ == query).len() # davinci F
def select_same_as(l, query): return seq(l).filter(_ == query) # simpler version of how_many. davinci F
def select_same_number_as(l, query): return seq(l).group_by(_).select(_[1]).filter(lambda x: len(x) == len(query)).flatten() # F
def includes(l, query): return seq(l).union(seq(query)).distinct().len() == seq(l).distinct().len() # davinci F
def is_included_by(l, query): return seq(l).difference(seq(query)).empty() # davinci F

In [23]:
def Compare_the_values_of_two_sets(cxt, query):
    s = cxt
    q = query
    if(seq(s).filter(lambda x: sl_vocab[x] == sl_vocab[q[0]]).len() > len(s)/2):
        return ">"
    elif(seq(s).filter(lambda x: sl_vocab[x] == sl_vocab[q[0]]).len() < len(s)/2):
        return "<"
    else:
        return "="

def Compare_sample(vocab, k):
    all_vocab,s_vocab,l_vocab = vocab
    k_cxt,k_query = k  # k_cxt = 5 ,k_query =2
    N = list(all_vocab.keys())
    s = sample(N, k_cxt) 
    return s
   
def Compare_query(cxt,vocab,k): 
    NA = cxt
    all_vocab,s_vocab,l_vocab = vocab
    k_cxt,k_query = k
    q = sample(s_vocab,1)+sample(l_vocab,1)
    random.shuffle(q)
    return q

def Compare_query2str(query):
    ss,q = query
    query_str = '%s,%s?' % (ss,q)
    return '. ' + query_str

In [24]:
def Relationship_between_two_sets(cxt, query): #新建联系，把vocab和属性连起来形成新集合
    NA = cxt
    s,q= query   #s:boys/girls  q:fruits/toys
    name = seq(NA).filter(lambda x: bg_vocab[x[0]] == s)
    if(seq(name).map(_[1]).filter(lambda x: ft_vocab[x] == q).len() == seq(name).len()):
        return "all"
    elif(seq(name).map(_[1]).filter(lambda x: ft_vocab[x] == q).empty()):
        return "none"
    else:
        return "some"
    
def Relationship_sample(vocab, k): 
    name_vocab, all_vocab = vocab
    k_name, k_all, k_SA = k  # k_name = 4, k_all = 4, k_SA = 4
    Name = list(name_vocab.keys())
    S = list(all_vocab.keys())
    N, A = sample(Name, k_name), sample(S, k_all)
#     D = sample(d := name_vocab.pop(name_vocab.index(a) for a in N),k_D)
    
    all_NA = list(itertools.product(N, A))  
    while seq(NA := sample(all_NA, k_SA)).map(_[0]).distinct().len() < k_name: continue  
    return NA
   
def Relationship_query(cxt,vocab,k): 
    NA = cxt
    name_vocab, all_vocab = vocab
    k_name, k_all, k_SA = k
    S,D = select_distinct(NA, 0),select_distinct(NA, 1)
    s,d = choice(S),choice(D)
    s1,d1 = name_vocab[s],all_vocab[d]
    return s1,d1

def Relationship_query2str(query):
    s,q = query
    query_str = '[] %s have %s. [all / some / none]?' % (s,q)
    return '. ' + query_str

In [25]:
tasks = [
    (ith_element,            None,                               sample,    None,None),
    (ith_group,              None, lambda vocab, k: seq(sample(vocab, k)).map(lambda x:[x]*randint(1, 3)).flatten().list(),None,None),
    (element_at_index,       lambda: upper_letters,              sample,    lambda l,vocab,k: randint(0, min(2,len(l)-1))),
    (replace,                None,                               sample,    lambda l,vocab,k: {choice(l): choice(vocab)}),
    (replace_with_the_other, lambda: sample(full_vocab, 2),   lambda vocab,k: sample(vocab+choices(vocab, k=k-2),k), None),
    (replace_all_with,       None,                               sample_rand_len, lambda l,vocab,k: choice(vocab)),
    (interleave_with,        None,                               sample_rand_len, lambda l,vocab,k: choice(vocab)),
    (unique_elements,        lambda: sample(upper_letters, 3),   choices,   None),
    (how_many_unique_elements,lambda: sample(upper_letters, 3),  choices,   None),
    (how_many,               lambda: sample(upper_letters, 3),   choices,   lambda l,vocab,k: choice(list(set(l)))),
    (select_same_as,         lambda: sample(upper_letters, 3),   choices,   lambda l,vocab,k: choice(list(set(l)))),
    (select_same_number_as,  None, lambda vocab, k: seq(sample(vocab, k)).map(lambda x:[x]*randint(1, 3)).flatten().list(),   
     lambda l,vocab,k: [choice(vocab)]*randint(1, 3)),
    (includes,               lambda: sample(upper_letters, 6),   sample,    lambda l,vocab,k: sample(vocab, 3)),
    (is_included_by,         lambda: sample(upper_letters, 6),   sample,    lambda l,vocab,k: sample(vocab, 5)),
    
    (Ascii_size_None,        lambda: string.ascii_uppercase,              sample,    lambda l,vocab,k: "Is there no element greater than %s?" % choice(list(set(l)))),
    (Ascii_size_all,         lambda: string.ascii_uppercase,              sample,    lambda l,vocab,k: "Are all elements greater than %s?" % choice(list(set(l)))),
    (Ascii_size_existence,   lambda: string.ascii_uppercase,              sample,    lambda l,vocab,k: "Is there an element greater than %s?" % choice(list(set(l)))),
    
#     (all_a,                  lambda: [names_vocab,courses_vocab,depts_vocab],     all_a_sample,    all_a_query, all_a_query2str),
    (Do_all_students_choose_courses_in_a_department,    lambda: [names_vocab,courses_vocab,depts_vocab],     all_a_sample,    all_a_query, all_a_query2str),
    (Compare_the_values_of_two_sets,  lambda: [sl_vocab,solid_vocab,liquid_vocab],  Compare_sample,    Compare_query, Compare_query2str),
    (Relationship_between_two_sets,   lambda: [bg_vocab,ft_vocab],               Relationship_sample,    Relationship_query, Relationship_query2str),
    (Is_the_intersection_of_two_sets_empty,           lambda: [string.ascii_uppercase,string.ascii_lowercase],     intersection_sample,    intersection_query, intersection_query2str),
    (Is_the_first_set_a_subset_of_the_second_one,           lambda: [string.ascii_uppercase,string.ascii_lowercase],     complement_sample,    complement_query, complement_query2str),
    (Are_they_the_union_of_the_last_element,                lambda: [string.ascii_uppercase,string.ascii_lowercase,depts_vocab],     union_sample,    union_query, union_query2str),
    (Are_there_elements_belonging_to_the_same_class,            lambda: [string.ascii_uppercase,sl_vocab],      find_sample,    find_query, find_query2str ),
    (Are_there_any_elements_different_from_other_elements,          lambda: [string.ascii_uppercase,sl_vocab],      find_dif_sample,  find_dif_query, find_dif_query2str ),
    (How_many_elements_are_similar_to_the_case,                lambda: sl_vocab,                                      count_sample,            count_query,count_query2str),
    (Is_the_number_of_first_elements_greater_than_the_second_one,              lambda: [sl_vocab,solid_vocab,liquid_vocab],          compare_sample,            compare_query,compare_query2str),
]

In [1365]:
# print(make_input_str(tasks[-8], nrows=30, ncols=(4, 4, 4))) 

In [664]:
# print(make_input_str(tasks[-9], nrows=8, ncols=(4,2))) #比较，query中给了两个元素（固体、液体），
                                                        #若前一个元素的类型数目大于后一个，就输出YES

In [665]:
# print(make_input_str(tasks[-9], nrows=80, ncols=(2, 3, 2, 5))) 

In [460]:
print(make_input_str(tasks[-3], nrows=80, ncols=(4,4,3))) 

Instruction: Are there any elements different from other elements. For example:
M: beer, N: pear, B: milk, G: peach. N, B and G? -> Yes
F: juice, P: cola, E: coffee, G: pineapple. F, P and E? -> No
U: grape, C: whisky, Z: juice, T: beer. Z, C and T? -> No
H: whisky, R: apple, L: strawberry, O: brandy. R, L and O? -> Yes
V: juice, W: brandy, P: coffee, O: banana. O, W and P? -> Yes
H: banana, J: pineapple, E: coffee, G: grape. H, G and J? -> No



In [462]:
print(make_input_str(tasks[-4], nrows=100, ncols=(3,4,2,4)))  #找相同

Instruction: Are there elements belonging to the same class. For example:
P: banana, P: brandy, I: brandy, U: peach; v: grape, b: wine. I,b? -> Yes
K: whisky, P: apple, K: cola, L: apple; v: whisky, a: strawberry. K,a? -> No
R: pear, R: cola, E: strawberry, M: cola; q: pear, h: vodka. E,h? -> No
N: pineapple, Z: pineapple, C: coffee, C: juice; p: lemon, j: milk. N,p? -> Yes
E: beer, J: milk, I: peach, I: beer; j: peach, n: banana. E,j? -> No
R: grape, P: grape, P: juice, N: grape; k: milk, n: vodka. P,k? -> Yes



In [668]:
# print(make_input_str(tasks[-1], nrows=8, ncols=(5,2))) #比较，query中给了两个元素（固体、液体），
                                                        #若前一个元素的类型数目大于后一个，就输出YES

In [669]:
# print(make_input_str(tasks[-2], nrows=30, ncols=(3,1), ans_vocab=[0,1,2,3]))    #数数，数cxt中与query中元素同类的个数
                                                                                  #这里修改了balance函数

In [670]:
# print(make_input_str(tasks[-5], nrows=10, ncols=(3,4,2,4)))

In [671]:
# print(make_input_str(tasks[-6], nrows=18, ncols=(3,11,5))) 

In [672]:
# print(make_input_str(tasks[-7], nrows=4, ncols=(3,4,5)))

In [1377]:
# print(make_input_str(tasks[-10], nrows=100, ncols=(3, 3, 2, 5))) 

Instruction: Do all students choose courses in a department. For example:
L: Geography, L: English, B: Chemistry, L: Chemistry, Z: Chemistry; English: Eng, Geography: Eng, Chemistry: Mat. B and L,Eng? -> No
J: History, O: Art, O: History, J: Math, D: History; History: CS, Art: Mat, Math: Mat. D and O,CS? -> Yes
Z: Sql, Z: Music, P: Chemistry, P: Sql, T: Music; Sql: Eng, Music: Eng, Chemistry: Chi. P and Z,Chi? -> No
J: Chinese, J: Music, U: Music, J: Sql, H: Chinese; Chinese: CS, Sql: Eng, Music: Eng. J and H,CS? -> Yes



In [674]:
# print(make_input_str(tasks[-2], nrows=8, ncols=4))

In [675]:
# if n_total == 1:
#     inputs = tokenizer.encode_plus(text, return_tensors='pt')
#     inputs = prepare_inputs(inputs, model.device)
#     outputs = model(**inputs, output_attentions=False)

#     # assert inputs.input_ids.size(0) == 1
#     input_ids = inputs.input_ids
#     logits = outputs.logits

#     bsz = input_ids.size(0); assert bsz == 1
#     labels = torch.ones_like(input_ids) * (-100)
#     for bi in range(bsz):
#         bos_indices = (input_ids[bi] == bos_id).nonzero().squeeze(1)
#         eos_indices = (input_ids[bi] == eos_id).nonzero()[-nrows:].squeeze(1)
#         for i, (example, bos_i, eos_i) in enumerate(zip(examples, bos_indices.tolist(), eos_indices.tolist())):
#             print(' ' + make_example_str(example))
#             ans_ids = input_ids[bi, bos_i + 1: eos_i]
#             if i >= 2: labels[bi, bos_i: eos_i - 1] = ans_ids
#             ans_prob_dist = logits[bi, bos_i: eos_i - 1].softmax(-1)
#             ans_probs = ans_prob_dist[torch.arange(ans_prob_dist.size(0)), ans_ids]
#             ans_tokens = tokenizer.convert_ids_to_tokens(ans_ids)
#             for ans_id, ans_token, ans_prob, dist in zip(ans_ids, ans_tokens, numpy(ans_probs, decimals=3), ans_prob_dist):
#                 top1_correct = (dist.argmax() == ans_id).item()
#                 print(('*' if top1_correct else ' ') + ans_token, ans_prob, 
#                       show_topk(*dist.topk(5), indices_fn=tokenizer.convert_ids_to_tokens)) 
#     loss = nn.CrossEntropyLoss()(logits.view(-1, logits.size(-1)), labels.view(-1))
#     loss

In [676]:
# tokenizer.convert_ids_to_tokens(617)

In [677]:
# sum(s.count('Yes') for s in input_strs)
# sum(s.count('No') for s in input_strs)
# sum(s.count('all') for s in input_strs)
# sum(s.count('none') for s in input_strs)
# sum(s.count('some') for s in input_strs)
# sum(s.count('3') for s in input_strs)

In [187]:
# [e['input_ids'] for e in train_dataset]

In [140]:
# tokenizer.convert_ids_to_tokens(([e['input_ids'] for e in train_dataset][0]).numpy().tolist())

In [141]:
# print(eval_dataset.labels)

# 请从这里开始,肖老师

In [297]:
# n_total, n_valid = 500, 100  #全部数目，测试数目
# n_total, n_valid = 800, 200
n_total, n_valid = 180, 30
# n_total, n_valid =4,2
n_train = n_total - n_valid
# input_strs = [make_input_str(tasks[-7], nrows=1, ncols=(3,4,5)) for __ in range(n_total)]# Is the intersection of two sets empty.
# input_strs = [make_input_str(tasks[-3], nrows=1, ncols=(4,4,3)) for __ in range(n_total)]# Are there any elements different from other elements
# input_strs = [make_input_str(tasks[-4], nrows=1, ncols=(3,4,2,4)) for __ in range(n_total)]# Are_there_elements_belonging_to_the_same_class
# input_strs = [make_input_str(tasks[-1], nrows=1, ncols=(5,2)) for __ in range(n_total)]#Is_the_number_of_first_elements_greater_than_the_second_one
# input_strs = [make_input_str(tasks[-2], nrows=1, ncols=(3,1), ans_vocab=[0,1,2,3]) for __ in range(n_total)] #How_many_elements_are_similar_to_the_example
# input_strs = [make_input_str(tasks[-5], nrows=1, ncols=(3,4,2,4)) for __ in range(n_total)] #Are_they_the_union_of_the_last_element
# input_strs = [make_input_str(tasks[-6], nrows=1, ncols=(3,11,5)) for __ in range(n_total)]#Is the first set a subset of the second one
# input_strs = [make_input_str(tasks[-10], nrows=1, ncols=(3, 3, 2, 5)) for __ in range(n_total)]
# input_strs = [make_input_str(tasks[-9], nrows=1, ncols=(4,2)) for __ in range(n_total)] #Compare the values of two sets.
input_strs = [make_input_str(tasks[-8], nrows=1, ncols=(4, 4, 4)) for __ in range(n_total)] #Relationship between two sets.

for s in sample(input_strs, 2): print(s)
# print(input_strs)

Instruction: Relationship between two sets. For example:
Daniel: lemon, Ashley: grape, Amanda: grape, Harrison: pear. [] boys have fruits. [all / some / none]? -> all

Instruction: Relationship between two sets. For example:
Sarah: lemon, Alex: peach, Harrison: grape, Eva: plasticine. [] boys have toys. [all / some / none]? -> none



In [113]:
# s,end = input_strs[0].index(":"),input_strs[0].index("For")
# name = input_strs[0][s+2:end]
# import json
# filename = './nrk/'+name
# with open(filename,"w") as file_obj:
#     json.dump(input_strs,file_obj)

In [311]:
for s in sample(text, 2): print(s)
n_total, n_valid = 180, 30
n_train = n_total - n_valid
text = text[:180]

Instruction: Is this sentence correct. For example:
A is less difficult to carry than B because A is smaller. Is that right? -> Yes

Instruction: Is this sentence correct. For example:
A is more dangerous to look at than B because A is less luminous. Is that right? -> No



In [323]:
# train_dataset = CHILDDataset(input_strs[:-n_valid], tokenizer)
# eval_dataset = CHILDDataset(input_strs[-n_valid:], tokenizer)
train_dataset = CHILDDataset(text[:-n_valid], tokenizer)
eval_dataset = CHILDDataset(text[-n_valid:], tokenizer)

Using pad_token, but it is not set yet.


In [324]:
wte = model.get_input_embeddings() #提取 embedding 中的 词向量部分
if hasattr(wte, 'wte'): wte = wte.wte  # already been wrapped
we = WrappedEmbedding(wte, prompt_len=40000)
model.set_input_embeddings(we) #为词向量赋值

In [325]:
def compute_metrics(pred):
    labels = []
    bos_indices = []
    eos_indices = []
    preds = []
    m = nn.Softmax(dim = -1)
    labels_loc = pred.label_ids.argmax(-1)
    for i in range(len(labels_loc)):
        labels.append(pred.label_ids[i][labels_loc[i]])
                                                                                 #     arraypre = pred.predictions[0] # 6B
    arraypre = pred.predictions                                                                              # 1.3B
    predss = arraypre.argmax(-1)
    num = ""
   
    for bi in range(predss.shape[0]):
        num = labels_loc[bi]
        preds.append(predss[bi, num-1:num]) 
        t = torch.from_numpy(pred.predictions[bi,num-1:num])                        #1.3B
                                                                                    #         t = torch.from_numpy(pred.predictions[0][bi,num-1:num])  #6B
        n = m(t)
        ids = torch.topk(n,3)[1].numpy().tolist()                                              #ids   /[0] 概率
        loc = torch.topk(n,3)[0].numpy().tolist()
                                                                                                 #         print(ids)
        ids = tokenizer.convert_ids_to_tokens(ids[0])
        loc = [float('{:.4f}'.format(i)) for i in loc[0]]
        precision = [i+" : "+str(j) for i,j in zip(ids,loc)]
    acc = accuracy_score(labels, list(preds))    
    return {
        'accuracy': acc,
        'precision': precision,
    }


In [326]:
eval_dataset[0]['labels']

tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, 3363, -100, -100, -100, -100,
        -100, -100])

In [316]:
# def compute_metrics(pred):
#     labels = []
#     bos_indices = []
#     preds = []
#     labels_loc = pred.label_ids.argmax(-1)
#     for i in range(len(labels_loc)):
#         labels.append(pred.label_ids[i][labels_loc[i]])
# #     print(labels)
#     arraypre = pred.predictions[0] # 6B
# # #     arraypre = pred.predictions # 1.3B
#     print(arraypre)
#     predss = arraypre.argmax(-1)
#     sent = tokenizer.convert_ids_to_tokens(predss[0])
#     sent1 = " ".join(sent)
#     sent1=sent1.replace("Ġ","")
#     sent1=sent1.replace("Ċ","\n")
#     print(sent1)
#     for bi in range(predss.shape[0]):
#         for j in range(predss.shape[1]):
#             if(predss[bi][j] == bos_id):
#                 bos_indices.append(j)
#         bos_i = bos_indices[-1]
#         preds.append(predss[bi, bos_i + 1:bos_i + 2])
    
#     acc = accuracy_score(labels, list(preds))
#     return {
# #         'accuracy': acc,
#     }


In [327]:
training_args = TrainingArguments(output_dir="./models/model_name",                                                             #模型预测和检查点的输出目录
    overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, 
    per_device_train_batch_size=1, per_device_eval_batch_size=1,                                                  #每个GPU / TPU内核/ CPU的批处理大小
    gradient_accumulation_steps=6,eval_steps=5, 
    weight_decay=0.001, adam_beta2=0.98, adam_epsilon=1e-6,                                      #weight_decay要应用的权重衰减,adam_epsilon AdamW优化器的ε超参数
    lr_scheduler_type='constant', learning_rate=0.001, num_train_epochs=5,                                  #learning_rate:Adam初始学习率
    logging_strategy ='epoch',  save_steps=0,                                             #save_steps保存两个检查点之前的更新步骤数
    no_cuda=True, report_to='none',                                                         # to avoid report to wandb
    evaluation_strategy ='steps',
#     evaluation_strategy ='epoch'
)

PyTorch: setting up devices


In [328]:
# with open(filename,"a") as f:
#         f.write(str(training_args))

In [329]:
trainer = Trainer(model, training_args, train_dataset=train_dataset, eval_dataset=eval_dataset,compute_metrics=compute_metrics,
                  optimizers=(create_optimizer(model, training_args), None))

/nas/xd/projects/transformers/src/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [330]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 30
  Batch size = 1


{'eval_loss': 3.5578513145446777,
 'eval_accuracy': 0.0,
 'eval_precision': ['ĠA : 0.1883', 'ĠIs : 0.059', 'ĠB : 0.0472'],
 'eval_runtime': 9.2024,
 'eval_samples_per_second': 3.26,
 'eval_steps_per_second': 3.26}

In [331]:
trainer.train()

***** Running training *****
  Num examples = 150
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 750


Epoch,Training Loss,Validation Loss,Accuracy,Precision
1,1.616900,1.304838,0.566667,"['ĠYes : 0.6459', 'ĠNo : 0.1877', 'Ġ[ : 0.0194']"
2,2.889400,13.196543,0.000000,"['ĊĊ : 0.1545', 'Ċ : 0.1327', '\\ : 0.0873']"
3,10.558700,12.672831,0.000000,"['Ġthe : 0.1758', 'Ġof : 0.0498', '- : 0.0434']"
4,9.712600,9.770533,0.000000,"['Ġ : 0.273', 'Ġthe : 0.0731', 'Ġlike : 0.035']"
5,4.668000,2.120934,0.433333,"['ĠNo : 0.1628', 'ĠYes : 0.1345', 'Ġthe : 0.0488']"


***** Running Evaluation *****
  Num examples = 30
  Batch size = 1
***** Running Evaluation *****
  Num examples = 30
  Batch size = 1
***** Running Evaluation *****
  Num examples = 30
  Batch size = 1
***** Running Evaluation *****
  Num examples = 30
  Batch size = 1
***** Running Evaluation *****
  Num examples = 30
  Batch size = 1


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=750, training_loss=5.889138916015625, metrics={'train_runtime': 554.7103, 'train_samples_per_second': 1.352, 'train_steps_per_second': 1.352, 'total_flos': 224246237184000.0, 'train_loss': 5.889138916015625, 'epoch': 5.0})

In [76]:
test_dataset = CHILDDataset(text[:10], tokenizer)
a = trainer.predict(test_dataset) #此处新生成了测试集
print(a.metrics)

***** Running Prediction *****
  Num examples = 10
  Batch size = 1


{'test_loss': 0.735599160194397, 'test_accuracy': 0.5, 'test_precision': ['ĠYes : 0.6578', 'ĠNo : 0.3009', 'Ġyes : 0.0071'], 'test_runtime': 3.1062, 'test_samples_per_second': 3.219, 'test_steps_per_second': 3.219}


In [101]:
# input_strs = [make_input_str(tasks[-7], nrows=100, ncols=(3,4,5)) for __ in range(30)]# Is the intersection of two sets empty.
# input_strs = [make_input_str(tasks[-3], nrows=100, ncols=(4,4,3)) for __ in range(30)]# Are there any elements different from other elements
# input_strs = [make_input_str(tasks[-4], nrows=100, ncols=(3,4,2,4)) for __ in range(30)]# Are_there_elements_belonging_to_the_same_class
# input_strs = [make_input_str(tasks[-1], nrows=120, ncols=(5,2)) for __ in range(30)]#Is_the_number_of_first_elements_greater_than_the_second_one
# input_strs = [make_input_str(tasks[-2], nrows=80, ncols=(3,1), ans_vocab=[0,1,2,3]) for __ in range(30)] #How_many_elements_are_similar_to_the_example
# input_strs = [make_input_str(tasks[-5], nrows=100, ncols=(3,4,2,4)) for __ in range(30)] #Are_they_the_union_of_the_last_element
# input_strs = [make_input_str(tasks[-6], nrows=100, ncols=(3,11,5)) for __ in range(30)]#Is the first set a subset of the second one
# input_strs = [make_input_str(tasks[-10], nrows=100, ncols=(3, 3, 2, 5)) for __ in range(30)]
# input_strs = [make_input_str(tasks[-9], nrows=100, ncols=(4,2)) for __ in range(30)] #Compare the values of two sets.
# input_strs = [make_input_str(tasks[-8], nrows=100, ncols=(4, 4, 4)) for __ in range(30)] #Relationship between two sets.
test_dataset = CHILDDataset(input_strs[:], tokenizer)
a = trainer.predict(test_dataset) #此处新生成了测试集
print(a.metrics)

***** Running Prediction *****
  Num examples = 30
  Batch size = 1


{'test_loss': 0.0003098619054071605, 'test_accuracy': 1.0, 'test_precision': ['ĠNo : 0.9999', 'No : 0.0', 'Ġ : 0.0'], 'test_runtime': 50.5283, 'test_samples_per_second': 0.594, 'test_steps_per_second': 0.594}


In [81]:
trainer.place_model_on_device

True

In [82]:
def get_prev(elem):
    i, v = elem
    return _l[i - 1] if i > 0 else None

false = lambda *_: False
true  = lambda *_: True

In [83]:
Element = namedtuple('Element', 'index value')
_l = 'A B C B'.split()
n = len(_l)
# l = [Element._make(e) for e in enumerate(l)]
l = seq(_l)
l = l.enumerate().map(Element._make)

In [85]:
l.map(lambda x: {'B': 'D'}.get(x, x))

l.filter(lambda x: get_prev(x) == 'B').select(_.value)

find_fn = _.index == 1
l.filter(find_fn).select(_.value).map(lower)

find_fn = _.value == 'C'
l.filter(find_fn).select(_.index)

# move x to first
update_filter = _.value == 'C'
get_new = lambda x: -1
l.map(lambda x: Element(update_fn(x, 'index'), x.value)).order_by(_.index).select(_.value)

# swap first and last
update_filter = true
get_new = lambda x: {0: n - 1, n - 1: 0}.get(x.index, x.index)
l.map(lambda x: Element(update_fn(x, 'index'), x.value)).order_by(_.index).select(_.value)

# get inbetween == drop_while + take_while?

# update by index to its prev
update_filter = _.index == 1
get_new = lambda x: get_prev(x)
def update_fn(x, update_field): return get_new(x) if update_filter(x) else getattr(x, update_field)
l.map(lambda x: Element(x.index, update_fn(x, 'value')))

# if two adjacent elements by indices are equal
l.filter(lambda x: x.index in [0, 1]).select(_.value).distinct().len() == 1

seq('A B C B C'.split()).group_by(_).select(_[1]).flatten()

# count occurance till current
seq('A B A C B A'.split()).inits().reverse().tail().map(lambda x: x.filter(_ == x.last()).len())

# find special
seq('A B A A'.split()).count_by_value().filter(_[1] == 1).select(_[0])

# generalized find special
seq('A A B C C D D'.split()).group_by(_).map(lambda x: (x[0], len(x[1]))).filter(_[1] == 1).select(_[0])

index,value
0,A
1,B
2,C
3,B


['C']

['b']

[2]

['C', 'A', 'B', 'B']

...

False

['A', 'B', 'B', 'C', 'C']

[1, 1, 2, 1, 2, 3]

['B']

['B']